# Creating a simple chatbot using the ChatGPT model

*__NOTE:__
For instructions on running the Jupyter Notebook that contains the labs see instructions here: <https://github.com/retaildevcrews/OpenAI-Labs>*

This sample notebook demonstrates  creation of a simple chatbot that uses the ChatGPT model using the Chat Completion API.  The API takes in an array of messages that have the following parts:
- role (system - sets the overall behavior for the assistant , user - provides specific instructions for the assistant, assistant - stores previous responses and can be used for fine tuning to achieve desired behavior)
- content

For this lab the system message will be a simple description, and the messages sent to the api will be a collection that inclues all of the previous messages and responses.  In addition this example will track the number of tokens in the message to ensure it does not exceed the maximum.

The chatbot labs are based on examples found in the <https://github.com/Azure/openai-samples> repository.


In [1]:
from ffmodel.core.inference_endpoint import InferenceEndpoint

environment_config_path = "~/.product-search.env"

#solution_config_path = "../solution/configs/00_baseline.yaml"
#solution_config_path = "../solution/configs/01_with_static_context_01.yaml"
#solution_config_path = "../solution/configs/02_with_fewshots_01 _min.yaml"
#solution_config_path = "../solution/configs/03_with_fewshots_01.yaml"
solution_config_path = "../solution/configs/05_with_fewshots_02_min.yaml"
#solution_config_path = "../solution/configs/04_with_fewshots_02.yaml"

endpoint = InferenceEndpoint(solution_config_path, environment_config_path)

2023-10-14 10:12:22,893 name=ffmodel.core.inference_endpoint level=INFO Initializing inference runtime. 
2023-10-14 10:12:22,894 name=ffmodel.utils.file_logger level=INFO File Logging is disabled: SolutionConfig and DataModel will not be logged 
2023-10-14 10:12:22,895 name=ffmodel.utils.file_logger level=INFO File Logging is disabled: SolutionConfig and DataModel will not be logged 
2023-10-14 10:12:22,895 name=ffmodel.core.inference_endpoint level=METRICS solution_config 
2023-10-14 10:12:22,896 name=ffmodel.core.inference_endpoint level=INFO Inference runtime initialized. 
2023-10-14 10:12:22,896 name=ffmodel.core.inference_endpoint level=INFO Downloading supporting data for solution pipeline. 
2023-10-14 10:12:22,900 name=ffmodel.core.inference_endpoint level=INFO Downloaded file names: [] 
2023-10-14 10:12:22,901 name=ffmodel.core.inference_endpoint level=INFO Initializing solution pipeline. 
2023-10-14 10:12:22,925 name=ffmodel.core.inference_endpoint level=INFO Adding reader com

Loading local environment configs from file.
Loaded 2 configs from file '/Users/jorgeluna/.product-search.env'.
Key vault url not set. Skipping key vault client initialization.
FFModelLogger: Logging will print to console as no AppInsight connection was provided (config `ApplicationInsightConnectionString)`.


2023-10-14 10:12:23,705 name=ffmodel.core.inference_endpoint level=INFO Adding component components.model_callers.openai_chat_completions to the pipeline. 
2023-10-14 10:12:23,713 name=ffmodel.core.inference_endpoint level=INFO Adding component components.post_processors.minify_json to the pipeline. 
2023-10-14 10:12:23,729 name=ffmodel.core.inference_endpoint level=INFO Skipping non-inference component components.evaluators.json_schema in the pipeline. 
2023-10-14 10:12:23,864 name=ffmodel.core.inference_endpoint level=INFO Skipping non-inference component components.evaluators.fuzzy in the pipeline. 
[nltk_data] Downloading package punkt to /Users/jorgeluna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-10-14 10:12:30,658 name=ffmodel.core.inference_endpoint level=INFO Skipping non-inference component components.evaluators.bleu in the pipeline. 
2023-10-14 10:12:31,057 name=ffmodel.core.inference_endpoint level=INFO Skipping non-inference component components.ev

## 1.0 Define Request to OpenAI API
For these labs we are using the Azure OpenAI Chat Completion API, this is different than the Completions API.  Information about its usage can be found here:
<https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions>
The chat completions API takes a collection of messages to provide history and context of chat to the model.  This function takes a collection of messages which are objects that define a role and content.

We also create a function that calculates the number of tokens received from the message collection so that we can keep track of message size and not exceed threshhold.


In [2]:
import tiktoken


# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(chat_request):
    response = endpoint.execute(chat_request)
    return response.model_output.completions[0]

# Defining a function that counts the number of tokens
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

## 2.0 The conversation

In [8]:
import json
import typing
import uuid

from pydantic import BaseModel
from fastapi.encoders import jsonable_encoder

class CompletionPair(BaseModel):
    user_nl:str
    completion:object

class UserSession(BaseModel):
    session_id:str
    prior_responses:list[CompletionPair]
    sequence: typing.Optional[int]


class ChatRequest(BaseModel):
    user_nl:str
    session:UserSession

# This is the first user message that will be sent to the model. Feel free to update this.
max_response_tokens = 500
token_limit=2048
name = input('Enter Your Name: ')
print ('Welcome to pizzaai.  What can I get for you?')
conversation=[]
session_id=str(uuid.uuid4())
message_count=0
while True:
    request = input(name+':')
    current_message={request}
    chat_request=ChatRequest(user_nl=request,session=UserSession(session_id=session_id,sequence=message_count,prior_responses=conversation))
    conv_history_token_count = num_tokens_from_messages(conversation)
    #reduce history to fall below limits
    while (conv_history_token_count+max_response_tokens >= token_limit):
        del conversation[1] 
        conv_history_token_count = num_tokens_from_messages(conversation)
    if request=="Bye" or request=='bye':
        print('Bot: Bye')
        break
    else:
        payload=json.dumps(jsonable_encoder(chat_request))
        response = send_message(payload)
        interaction = CompletionPair(user_nl=request,completion=response)
        conversation.append(interaction)
        message_count+=1
        print('sent token count: ' + str(conv_history_token_count))
        print('Bot: ', response)

Welcome to pizzaai.  What can I get for you?


2023-10-14 10:20:14,860 name=ffmodel.core.inference_endpoint level=INFO Starting to process user prompt: user_nl='I would like some candy' session=UserSession(session_id='95333923-cae4-4c6f-8f79-03b9202489d8', prior_responses=[], sequence=0) 
2023-10-14 10:20:14,861 name=ffmodel.core.inference_endpoint level=INFO Executing components.pre_processors.static_context_from_file 
2023-10-14 10:20:14,862 name=ffmodel.core.inference_endpoint level=INFO Executing components.pre_processors.few_shots_from_file 
2023-10-14 10:20:14,862 name=ffmodel.core.inference_endpoint level=INFO Executing components.stitchers.openai_chat_completions 
2023-10-14 10:20:14,863 name=ffmodel.core.inference_endpoint level=ERROR components.stitchers.openai_chat_completions Component failed with an exception Object of type ChatRequest is not JSON serializable. 
2023-10-14 10:20:14,863 name=ffmodel.core.inference_endpoint level=ERROR Object of type ChatRequest is not JSON serializable 
Traceback (most recent call last)

TypeError: Object of type ChatRequest is not JSON serializable